# _Проекту 4. Компьютер говорит «Нет»_

**Юнит 5. Основные алгоритмы машинного обучения. Часть I (Andrew Glybin)**

## Основная информация

Вашей задачей будет построить скоринг модель для вторичных клиентов банка, которая бы предсказывала вероятность дефолта клиента. Для этого нужно будет определить значимые параметры заемщика.

Вам предоставлена информация из анкетных данных заемщиков и факт наличия дефолта.

**Описания полей:**

* client_id - идентификатор клиента

* education - уровень образования

* sex - пол заемщика

* age - возраст заемщика

* car - флаг наличия автомобиля

* car_type - флаг автомобиля иномарки

* decline_app_cnt - количество отказанных прошлых заявок

* good_work - флаг наличия “хорошей” работы

* bki_request_cnt - количество запросов в БКИ

* home_address - категоризатор домашнего адреса

* work_address - категоризатор рабочего адреса

* income - доход заемщика

* foreign_passport - наличие загранпаспорта

* sna - связь заемщика с клиентами банка

* first_time - давность наличия информации о заемщике

* score_bki - скоринговый балл по данным из БКИ

* region_rating - рейтинг региона

* app_date - дата подачи заявки

* default - флаг дефолта по кредиту

**Метрика качества**

Результаты оцениваются по площади под кривой ROC AUC

## 1. Импорт библиотек 


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# PATH_to_file = '/kaggle/input/sf-dst-scoring/'

In [ ]:
from pandas import Series
from datetime import datetime, timedelta

from sklearn.feature_selection import f_classif, mutual_info_classif
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.preprocessing import PolynomialFeatures

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import confusion_matrix
from sklearn.metrics import auc, roc_auc_score, roc_curve
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import plot_confusion_matrix, plot_roc_curve, plot_precision_recall_curve

import warnings
warnings.filterwarnings('ignore')

import pandas_profiling

import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline

In [ ]:
# изменим параметры для изображений
sns.set_context(
    "notebook", 
    font_scale=1.5,       
    rc={ 
        "figure.figsize": (11, 8), 
        "axes.titlesize": 18 
    }
)

# графики в svg выглядят более четкими
%config InlineBackend.figure_format = 'svg' 

# увеличим дефолтный размер графиков
from pylab import rcParams
# rcParams['figure.figsize'] = 8, 5
# from matplotlib import rcParams
rcParams['figure.figsize'] = 11, 8

pd.set_option('display.max_columns', None)
np.set_printoptions(precision=2)


In [ ]:
# зафиксируем RANDOM_SEED, версию пакетов, общую текущую дату
RANDOM_SEED = 42
!pip freeze > requirements.txt
CURRENT_DATE = pd.to_datetime('06/11/2020')

## 2. Импорт данных

In [ ]:
# df_train = pd.read_csv(PATH_to_file+'train.csv')
# df_test = pd.read_csv(PATH_to_file+'test.csv')
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

print('Размерность тренировочного датасета: ', df_train.shape)
print('Размерность тестового датасета: ', df_test.shape)

# sample_submission = pd.read_csv(PATH_to_file+'sample_submission.csv')

In [ ]:
df_train.head(1)

In [ ]:
df_test.head(1)

In [ ]:
# ВАЖНО! дря корректной обработки признаков объединяем трейн и тест в один датасет

df_train['sample'] = 1   # помечаем где у нас трейн
df_test['sample'] = 0    # помечаем где у нас тест
df_test['default'] = -1  # в тесте у нас нет значения default, мы его должны предсказать, 
                         # но его значения 0 или 1, поэтому заполняем его временно -1 для избежания ошибки

data = df_test.append(df_train, sort=False).reset_index(drop=True)   # объединяем

## 3. Предварительный анализ данных

In [ ]:
pandas_profiling.ProfileReport(df_train)

In [ ]:
df_train.info()

In [ ]:
df_test.info()

In [ ]:
data.info()

In [ ]:
(data.isna()).sum()

In [ ]:
data['region_rating'].value_counts()

### Резюме по предварительному анализу данных: 

В тренировочной выборке 73799 клиентов, в тестовой выборке 36349. 

Всего имеются данные о 110148 клиентах. 

Всего представлены 20 признаков, из них 1 - временной ряд, 7 бинарных (в т.ч. добавленный признак ***sample***), 6 категориальных и 6 числовых. 

Всего пропусков 478 (307 - в наборе train, 171 - в наборе test), все пропуски в переменной ***education***. 

***client_id*** уникальный числовой признак, который содержит числовой признак клиента. Вероятно, не несет полезной информации для модели. 

Целевая переменная ***default*** является бинарной (True - False).

Сильной корреляции между численными признаками не наблюдается ни в одной из матриц корреляции.

In [ ]:
# создаём списки на основании резюме
# client_id, default, sample не включаем в списки

# временной ряд (1)
time_cols = ['app_date']

# бинарные признаки (7-2)
bin_cols = ['sex', 'car', 'car_type', 'good_work', 'foreign_passport']

# категориальные признаки (6)
cat_cols = ['education', 'region_rating', 'home_address', 'work_address', 'sna', 'first_time']

# числовые признаки (6-1)
num_cols = ['age', 'decline_app_cnt', 'score_bki', 'bki_request_cnt', 'income']

# default признаём целевой переменной 
target = 'default'

# ID клиентов удаляем из-за неинформативности
data.drop(['client_id'], axis=1, inplace=True)

## 4. Функции для анализа данных

In [ ]:
def outliers(df_col):
    '''
    Определение наличия выбросов в отдельном признаке датасета.
    Расчёт данных описательной статистики. Определение границ выбросов.
    Подсчёт общего количества выбросов в признаке.
    
    df_col - название столбца датасета. 
    
    '''
    
    q1 = df_train[df_col].quantile(0.25)
    q3 = df_train[df_col].quantile(0.75)
    IQR = q3 - q1
    low = q1 - (1.5 * IQR)
    high = q3 + (1.5 * IQR)
    for i in df_train[df_col]:
        if (i <= low) or (i >= high):
            print("В признаке '{}' есть значения, которые могут считаться выбросами".format(df_col))
            break
        
    print("Количество выбросов в признаке '{}': {}."
          .format(df_col, ((df_train[df_col] < (q1 - 1.5 * IQR)) | (df_train[df_col] > (q3 + 1.5 * IQR))).sum()))
    print("25-й процентиль: {}, 75-й процентиль: {}, IQR: {}, Границы выбросов: [{}, {}]".format(q1, q3, IQR, low, high))
    

In [ ]:
def numerical_features(item_name, n_bins=100):
    '''
    Функция для анализа численных признаков.
    
    Рисует следующие графики:
    - боксплот `sns.boxplot()`: удобно выявлять аномалии
    - распределение `sns.distplot()`
     
    Вычисляет базовые статистические показатели `.describe()` 
        
    item_name - название численного признака датасета.
    
    '''
    
    fig, axes = plt.subplots(1, 2, figsize=(15, 10))
        
    # boxplot    
    df_train.boxplot(column=item_name, ax=axes[0])
    axes[0].set_title(item_name, fontsize=22)
    
    # distplot
    sns.distplot(df_train[item_name], bins=n_bins, kde=False, ax=axes[1], vertical=True)
    axes[1].set_title(item_name, fontsize=22)
    plt.xticks(rotation=45)
         
    # descriptive statistics
    print(pd.DataFrame(df_train[item_name].describe()).T, "\n")


In [ ]:
def split(df):
    '''
    Функция для разбиения датасета на тренировочный и валидационный
    
    '''
    
    y = df.default.values            
    x = df.drop(columns=['default'])
    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=RANDOM_SEED)
    return X_train, X_test, y_train, y_test


In [ ]:
def all_metrics(y_true, y_pred, y_pred_prob):
    '''
    Функция выводит в виде датафрейма значения основных метрик классификации
    
    '''
    
    dict_metric = {}
    P = np.sum(y_true==1)
    N = np.sum(y_true==0)
    TP = np.sum((y_true==1)&(y_pred==1))
    TN = np.sum((y_true==0)&(y_pred==0))
    FP = np.sum((y_true==1)&(y_pred==0))
    FN = np.sum((y_true==0)&(y_pred==1))
    
    dict_metric['P'] = [P,'Наличие дефолта']
    dict_metric['N'] = [N,'Отсутствие дефолта']
    dict_metric['TP'] = [TP,'Верно принято']
    dict_metric['TN'] = [TN,'Верно отвергнуто']
    dict_metric['FP'] = [FP,'Ошибка первого рода']
    dict_metric['FN'] = [FN,'Ошибка второго рода']
    dict_metric['Accuracy'] = [accuracy_score(y_true, y_pred),'Доля верно определённых']
    dict_metric['Precision'] = [precision_score(y_true, y_pred),'Точность определения'] 
    dict_metric['Recall'] = [recall_score(y_true, y_pred),'Полнота определения']
    dict_metric['F1-score'] = [f1_score(y_true, y_pred),'Гармоническое среднее Precision и Recall']
    dict_metric['ROC_AUC'] = [roc_auc_score(y_true, y_pred_prob),'Площадь под кривой ошибок']    

    temp_df = pd.DataFrame.from_dict(dict_metric, orient='index', columns=['Значение', 'Описание'])
    display(temp_df)   
    

In [ ]:
def show_confusion_matrix(lastmodel):
    '''
    Функция отображает пару confusion-матриц:
    с абсолютными значениями и нормализованную.
    
    На вход подаётся последняя модель.
    Функция использует тестовые значения выборки и целевого признака.
    
    '''
    
    class_names = ['NonDefault', 'Default']
    titles_options = [("Confusion matrix без нормализации", None),
                      ("Нормализованная confusion matrix", 'true')]
    for title, normalize in titles_options:
        disp = plot_confusion_matrix(lastmodel, X_test, y_test, 
                                     display_labels=class_names, 
                                     cmap=plt.cm.Blues, 
                                     normalize=normalize)
        disp.ax_.set_title(title)

        print(title)
        print(disp.confusion_matrix)

    plt.show()
    

In [ ]:
def show_roc_curve(lastmodel):
    '''
    Функция отображает ROC-кривую.
    
    На вход подаётся последняя модель.
    Функция использует тестовые значения выборки и целевого признака.
    
    '''
    
    probs = lastmodel.predict_proba(X_test)
    probs = probs[:,1]
    
    fpr, tpr, threshold = roc_curve(y_test, probs)
    roc_auc = roc_auc_score(y_test, probs)
    
    fpr, tpr, _ = roc_curve(y_test, y_pred_prob)
    plt.figure()
    plt.plot([0, 1], label = 'Случайный классификатор', linestyle='--')
    plt.plot(fpr, tpr, label = 'Логистическая регрессия')
    plt.title('Логистическая регрессия ROC AUC = %0.3f' % roc_auc)
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    plt.legend(loc = 'lower right')
    plt.show()
    

## 5. Анализ датасета по признакам.

### 5.1 Численные признаки:

In [ ]:
for col in num_cols:
    outliers(col)
    numerical_features(col)

#### Резюме по численным признакам:

После построения гистограмм стало очевидно, что распределения почти всех числовых переменных имеют тяжёлый правый хвост. 

**age**: Распределение логнормальное, выбросов по квартилям нет. Признак будем использовать, как есть.

**decline_app_cnt**: Распределение логнормальное, выбросов очень много. Возьмём логарифм, чтобы избежать чувствительности к сильным отклонениям. Выбросы удалять пока не будем. Возможно это потребуется для улучшения модели.

**score_bki**: Распределение нормальное. Выбросов не много, удалять не будем.

**bki_request_cnt**: Распределение логнормальное, выбросов не много, удалять не будем. Возьмём логарифм, чтобы сделать распределение более нормальным.

**income**: Распределение логнормальное, выбросов очень много, удалять их пока не будем. Возьмём логарифм. К вопросу об удалении выбросов вернемся после построения модели при необходимости.

In [ ]:
# логорифмируем "плохие" показатели
for i in ['decline_app_cnt', 'bki_request_cnt', 'income']:
    data[i] = np.log(data[i] + 1)
   

In [ ]:
# построим повторно графики для скорректированных показателей
for i in ['decline_app_cnt', 'bki_request_cnt', 'income']:
    plt.figure()
    sns.distplot(data[i][data[i] > 0].dropna(), kde = False, rug=False)
    plt.title(i)
    plt.show()

Признак ***income*** стал менее смещённым. Выбросов стало меньше во всех скорректированных признаках. Однако признаки ***decline_app_cnt*** и ***bki_request_cnt*** требуют дополнительного анализа.

### 5.2 Преобразование временного ряда.


In [ ]:
data['app_date'] = pd.to_datetime(data['app_date'], format='%d%b%Y')
data.head(3)

In [ ]:
# проверим начало и конец периода нашего датасета 
start = data.app_date.min()
end = data.app_date.max()
start, end

Представленный датасет охватывает период с 1 января по 30 апреля 2014 года. Можно создать дополнительный численный признак, отражающий количество дней от старта датасета до конкретной записи. Это необходимо сделать, т.к. собственно сам временной ряд необходимо будет позже исключить из датасета перед построением модели.

In [ ]:
data['app_date_delta'] = (data.app_date - start).dt.days.astype('int')

In [ ]:
# проверим количество дефолтов в течение срока, отражённого в представленном датасете
data_temp = data.loc[data['sample'] == 1] 
data_temp[['default'] + ['app_date_delta']].groupby('app_date_delta').sum().plot()

#### **ВАЖНО!** 
Количество дефолтов с ориентировочно 90 дня от начала датасета начало снижаться. Предположительно, были введены некие заградительные меры от выдачи рискованных кридетов и, очевидно, что это необходимо будет учитывать при построении модели, т.к. её поведение неоднородно в течение анализируемого периода.

In [ ]:
# добавление нового признака в список числовых признаков
num_cols.append('app_date_delta')

# удаление временного ряда из датасета
data.drop(['app_date'], axis=1, inplace=True)

### 5.3 Оценка значимости непрерывных переменных.

In [ ]:
imp_num = Series(f_classif(data_temp[num_cols], data_temp['default'])[0], index = num_cols)
imp_num.sort_values(inplace = True)
imp_num.plot(kind = 'barh')

#### Резюме по оценке значимости числовых переменных:
Скорринговый балл БКИ (**score_bki**) является самым значимым показателем согласно однофакторного дисперсионного анализа, менее значимым является кол-во отказанных ранее заявок (**declain_app_cnt**). Остальные признаки не оказывают существенного влияния на целевой показатель. Признак decline_app_cnt имеет очень большое количество выбросов и "плохое" распределения для модели. Нужно будет провести дополнительную работу.

### 5.4 Категориальные и бинарные признаки:

В ходе предварительного анализа данных было выявлено наличие пропусков только в одном признаке - **education**. 
Устраним эти пропуски, заполним значением IDK (неизвестно).   
Кроме того, имеется информация, что категории в данном признаке имеют следующие значение:
* SCH    (школа)
* GRD    (бакалавр)
* UGR    (кандидат в бакалавры)
* PGR    (магистр)
* ACD    (академик)

In [ ]:
data.education.fillna('IDK', inplace=True)   

In [ ]:
data.education.value_counts()

In [ ]:
# оценим их распределение визуально
sns.boxplot(x="age", y="education", data=data, whis=[0, 100], width=.6, palette="vlag")

Очевидно, что пропущенные значения (отмечены IDK) распределены почти аналогично с клиентами, имеющими только базовое образование. 
Преобразовать признак "education" в числовой формат можно с помощью LabelEncoder, но  тогда будет потерян вес образования, который, возможно, имеет влияние на целевую переменную.
Поэтому, предлагается провести преобразование с помощью словаря, назначив "веса" различным типам образования.
    

In [ ]:
data.education = data.education.replace({'IDK': 0, 'SCH': 1, 'UGR': 2, 'GRD': 3, 'PGR': 4, 'ACD': 5})

In [ ]:
# перевод бинарных признаков в числовой формат

label_encoder = LabelEncoder()

for col in bin_cols:
    data[col] = label_encoder.fit_transform(data[col])

In [ ]:
data.sample(3)

### 5.5 Оценка значимости категориальных и бинарных переменных.

In [ ]:
# для бинарных и категориальных признаков (переведенных в числа)
data_temp = data.loc[data['sample'] == 1]
imp_cat = Series(mutual_info_classif(data_temp[bin_cols + cat_cols], data_temp['default'],
                                     discrete_features =True), index = bin_cols + cat_cols)
imp_cat.sort_values(inplace = True)
imp_cat.plot(kind = 'barh')

#### Резюме по оценке значимости категориальных и бинарных переменных:
Самым значимым признаком по Mutual information тесту является связь заемщика с клиентами банка (**sna**). Далее, давность наличия информации о заемщике (**first_time**), а затем, практически с одинаковым значением, идет рейтинг региона (**region_rating**) и категоризатор домашнего адреса (**home_address**). Ещё одним достаточно значимым для модели признаком явялется уровень образования клиента (**education**)

## 6. Подготовка данных к машинному обучению.

### Определение дисбаланса классов

In [ ]:
sns.countplot(data_temp['default'])

In [ ]:
data_temp['default'].value_counts()

Несмотря на значительное различие по количеству дефолтных и надёжных клиентов, явного дисбаланса классов нет. Согласно теории, когда мы используем вероятностные модели для бинарной классификации, во время обучения модели не сильно зависят от баланса классов, а при тестировании будет использоваться метрика ROC_AUC, обладающая низкой чувствительностью к балансу классов. 

### Применение dummy-кодированием для категориальных переменных

In [ ]:
data = pd.get_dummies(data, prefix=cat_cols, columns=cat_cols)

### Стандартизация числовых признаков

In [ ]:
data[num_cols] = pd.DataFrame(StandardScaler().fit_transform(data[num_cols]), columns = data[num_cols].columns)

### Верификация случайных строк датасета

In [ ]:
display(data.sample(2))

### Выделение тренировочной и тестовой частей датасета

In [ ]:
train_data = data.query('sample == 1').drop(['sample'], axis=1)
test_data = data.query('sample == 0').drop(['sample'], axis=1)

# y = train_data.default.values            # наш таргет
# X = train_data.drop(['default'], axis=1)

## 7. Построение модели

### Разбиение тренировочного датасета на тренировочную и валидационную части


In [ ]:
X_train, X_test, y_train, y_test = split(train_data)

In [ ]:
# проверяем
test_data.shape, train_data.shape, X_train.shape, X_test.shape

### Обучаем модель, генерируем результат и сравниваем с тестом

In [ ]:
model = LogisticRegression(random_state=RANDOM_SEED)

model.fit(X_train, y_train)

y_pred_prob = model.predict_proba(X_test)[:,1]
y_pred = model.predict(X_test)

In [ ]:
all_metrics(y_test, y_pred, y_pred_prob)
show_roc_curve(model)
show_confusion_matrix(model)

### Выводы по результатам работы первой модели

Несмотря на относительно высокий показатель ROC_AUC, модель показывает абсолютно неудовлетворительную работу. Об этом свидетельствует очень низкий показатель количества правильных предсказаний от всего класса истинных значений (Recall). Т.о. велика вероятность дефолта по выданным кредитам и прямая потеря банком ресурсов. 

### Поиск оптимальных параметров модели

In [ ]:
# best practice by @Анна Нохрина
# запускаем GridSearch на небольшом кол-ве итераций max_iter=50 и с достаточно большой дельтой останова tol1e-3
# чтобы получить оптимальные параметры модели в первом приближении
model = LogisticRegression(random_state=RANDOM_SEED)

iter_ = 50
epsilon_stop = 1e-3

param_grid = [
    {'penalty': ['l1'], 
     'solver': ['liblinear', 'lbfgs'], 
     'class_weight':['none', 'balanced'], 
     'multi_class': ['auto','ovr'], 
     'max_iter':[iter_],
     'tol':[epsilon_stop]},
    {'penalty': ['l2'], 
     'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'], 
     'class_weight':['none', 'balanced'], 
     'multi_class': ['auto','ovr'], 
     'max_iter':[iter_],
     'tol':[epsilon_stop]},
    {'penalty': ['none'], 
     'solver': ['newton-cg', 'lbfgs', 'sag', 'saga'], 
     'class_weight':['none', 'balanced'], 
     'multi_class': ['auto','ovr'], 
     'max_iter':[iter_],
     'tol':[epsilon_stop]},
]
gridsearch = GridSearchCV(model, param_grid, scoring='f1', n_jobs=-1, cv=5)
gridsearch.fit(X_train, y_train)
modelbest = gridsearch.best_estimator_

# вывод параметров
best_parameters = modelbest.get_params()
for param_name in sorted(best_parameters.keys()):
    print('\t%s = %r' % (param_name, best_parameters[param_name]))
   

In [ ]:
# печатаем метрики
preds = modelbest.predict(X_test)
print('Accuracy: %.4f' % accuracy_score(y_test, preds))
print('Precision: %.4f' % precision_score(y_test, preds))
print('Recall: %.4f' % recall_score(y_test, preds))
print('F1: %.4f' % f1_score(y_test, preds))

#### Резюме:
Полученный результат показывает значительный рост метрик полноты и f1_score.
Есть надежда, что модель будет работать лучше.


### Построение модели после первой оптимизации

In [ ]:
model_2 = LogisticRegression(random_state=RANDOM_SEED, 
                           C=1, 
                           class_weight = 'balanced', 
                           dual = False, 
                           fit_intercept = True, 
                           intercept_scaling = 1, 
                           l1_ratio = None, 
                           multi_class = 'auto', 
                           n_jobs = None, 
                           penalty = 'l1', 
                           solver = 'liblinear', 
                           tol = 0.001, 
                           verbose = 0, 
                           warm_start = False)

model_2.fit(X_train, y_train)

y_pred_prob = model_2.predict_proba(X_test)[:,1]
y_pred = model_2.predict(X_test)

### Оценка качества оптимизированной модели

In [ ]:
all_metrics(y_test, y_pred, y_pred_prob)
show_roc_curve(model_2)
show_confusion_matrix(model_2)

In [ ]:
plot_precision_recall_curve(model_2, X_test, y_test)

### Выводы по результатам первой оптимизации модели

Качество работы модели значительно улучшилось. Её работа позоляет предсказать почти 70% возможных дефолтов по выданным кредитам. Однако модель отказывает трети надёжных клиентов, что потенциально приведёт к недополучению банком прибыли. 

## 8. Submission

Восстановим повторно тренировочный и тестовый датафреймы из объединённого датасета.**

In [ ]:
data.sample(2)

In [ ]:
train_data = data.query('sample == 1').drop(['sample'], axis=1)
test_data = data.query('sample == 0').drop(['sample'], axis=1)

In [ ]:
# назначение тренировочной и тестовой части выборки
X_train = train_data.drop(['default'], axis=1)
y_train = train_data.default.values
X_test = test_data.drop(['default'], axis=1)

In [ ]:
X_test.head(3)

In [ ]:
# sample_submission

In [ ]:
predict_submission = model_2.predict_proba(X_test)[:,1]

In [ ]:
submission = pd.DataFrame(df_test.client_id)
submission['default'] = predict_submission
submission.to_csv('submission.csv', index=False)

In [ ]:
submission

## Выводы

Модель требует доработки с т.ч. избавления от неинформативных признаков, возможны экперименты с созданием полиноминальных признаков.
Неободимо убрать выбросы в существующих признаках и оценить изменения оценки качества новой модели.
Однако, в рамках 48-формата выполнения проекта, я просто не рештился на вышеназванные эксперименты с признаками.
Отдельное спасибо Анне за предоставленный фрагмент кода для поиска оптимальных гиперпараметров модели. 